In [45]:
import pandas as pd
import numpy as np

input_file = "input.txt"  # 원본 파일명
output_file = r"C:\Users\dinle\iCloudDrive\문서\코인\뉴.xlsx"
all_cols = ["거래일시", "자산", "거래구분", "거래수량", "체결가격", "거래금액", "수수료", "정산금액", "상태"]
num_cols = ["거래수량", "체결가격", "거래금액", "수수료", "정산금액"]

def safe_float_convert(x):
    x = str(x)
    return float(x.replace(",", "")) if x != "-" else x

In [46]:
# 읽기
with open(input_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip() != "완료"]  # 빈 줄 제거
    
df_intput = pd.DataFrame(np.array(lines).reshape(-1, 14))
df_intput = df_intput.drop(columns=[df_intput.columns[2], df_intput.columns[5], df_intput.columns[7], df_intput.columns[9], df_intput.columns[11]])
df_intput.columns = all_cols
df_intput["상태"] = np.nan
df_intput["거래일시"] = pd.to_datetime(df_intput["거래일시"])

df_intput

,거래일시,자산,거래구분,거래수량,체결가격,거래금액,수수료,정산금액,상태
0,2023-10-20 19:53:45,원화,입금,"300,000",-,"300,000",-,"+300,000",NaN


In [47]:
df_org = pd.read_excel(output_file)
df_org

,거래일시,자산,거래구분,거래수량,체결가격,거래금액,수수료,정산금액,상태
0,2023-10-20 19:53:45,원화,입금,300000,-,300000,-,300000,NaN
1,2023-10-20 19:53:45,원화,입금,300000,-,300000,-,300000,NaN
2,2023-10-21 21:41:06,원화,출금,298483,-,297483,1000,-298483,NaN
3,2023-10-31 14:40:39,원화,입금,2500000,-,2500000,-,2500000,NaN
4,2023-11-02 19:22:16,원화,출금,103172,-,102172,1000,-103172,NaN
...,...,...,...,...,...,...,...,...,...
402,2025-02-27 20:41:57,원화,출금,48259,-,47259,1000,-48259,NaN
403,2025-02-28 06:32:31,원화,포인트샵 입금,30800,-,30800,-,30800,NaN
404,2025-03-01 02:47:27,원화,출금,97902,-,96902,1000,-97902,NaN
405,2025-03-01 16:43:39,원화,출금,91784,-,90784,1000,-91784,NaN


In [48]:
# 병합
df_output = pd.concat([df_org, df_intput], ignore_index=True)
df_output[num_cols] = df_output[num_cols].applymap(safe_float_convert)
df_output = df_output.drop_duplicates().dropna(subset=[df_org.columns[0]]) 
df_output = df_output.sort_values(by="거래일시", ascending=True)

df_output

,거래일시,자산,거래구분,거래수량,체결가격,거래금액,수수료,정산금액,상태
0,2023-10-20 19:53:45,원화,입금,300000.0,-,300000.0,-,300000.0,NaN
2,2023-10-21 21:41:06,원화,출금,298483.0,-,297483.0,1000.0,-298483.0,NaN
3,2023-10-31 14:40:39,원화,입금,2500000.0,-,2500000.0,-,2500000.0,NaN
4,2023-11-02 19:22:16,원화,출금,103172.0,-,102172.0,1000.0,-103172.0,NaN
5,2023-11-09 19:30:07,원화,출금,171570.0,-,170570.0,1000.0,-171570.0,NaN
...,...,...,...,...,...,...,...,...,...
402,2025-02-27 20:41:57,원화,출금,48259.0,-,47259.0,1000.0,-48259.0,NaN
403,2025-02-28 06:32:31,원화,포인트샵 입금,30800.0,-,30800.0,-,30800.0,NaN
404,2025-03-01 02:47:27,원화,출금,97902.0,-,96902.0,1000.0,-97902.0,NaN
405,2025-03-01 16:43:39,원화,출금,91784.0,-,90784.0,1000.0,-91784.0,NaN


In [49]:
# 쓰기

with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df_output.to_excel(writer, index=False)  # 기존 데이터 + 새로운 데이터 저장